In [3]:
#!/usr/bin/env python3
import json
from collections import defaultdict

def tipo_valor(v):
    if v is None:
        return "null"
    if isinstance(v, bool):
        return "bool"
    if isinstance(v, int) and not isinstance(v, bool):
        return "int"
    if isinstance(v, float):
        return "float"
    if isinstance(v, str):
        return "str"
    if isinstance(v, list):
        return "list"
    if isinstance(v, dict):
        return "dict"
    return type(v).__name__

def recorrer(obj, path, skeleton):
    t = tipo_valor(obj)
    skeleton[path]['types'][t] += 1
    skeleton[path]['count'] += 1

    if isinstance(obj, dict):
        for k, v in obj.items():
            newp = f"{path}.{k}" if path else k
            recorrer(v, newp, skeleton)
    elif isinstance(obj, list):
        list_path = f"{path}[]" if path else "[]"
        for item in obj:
            recorrer(item, list_path, skeleton)

def extraer_esqueleto(file_in, file_out):
    with open(file_in, "r", encoding="utf-8") as f:
        data = json.load(f)

    skeleton = defaultdict(lambda: {"types": defaultdict(int), "count": 0})

    if isinstance(data, list):
        for juego in data:
            recorrer(juego, "", skeleton)
    elif isinstance(data, dict):
        recorrer(data, "", skeleton)

    # Convertir tipos a dict normal para JSON
    out = {k: {"types": dict(v["types"]), "count": v["count"]} for k,v in skeleton.items()}

    with open(file_out, "w", encoding="utf-8") as f:
        json.dump(out, f, ensure_ascii=False, indent=2)

# Uso
extraer_esqueleto("bgg_corpus.json", "esqueleto.json")